In [72]:
import subprocess
from tempfile import NamedTemporaryFile

from rnapolis import parser, annotator, tertiary

from contact_utils import ResiduesSelect, structure_cif_filter
in_contact= {'A172', 'A415', 'T2', 'A319', 'A96', 'P2', 'A366', 'A386', 'P1', 'A367', 'A168', 'A604', 'A601', 'A97', 'A141', 'T4', 'A283', 'A318', 'T3'}
structure3d=''
with NamedTemporaryFile(suffix="_changed.cif") as cif_rnapolis:
    p1=subprocess.Popen('awk \' {if (NF > 3) {$7 = $17; $9 = $16; print; } else { print; }}\' < /home/adamczykb/projects/RNAsolo_neighbour/tmp8t1ieku9_RNA.cif' ,
                        stdout=subprocess.PIPE,shell=True)
    cif_rnapolis.write(p1.communicate()[0])
    # structure_cif_filter('/home/adamczykb/projects/RNAsolo_neighbour/tmpqb7hfx3d_RNA.cif', ResiduesSelect(in_contact))
    structure3d = parser.read_3d_structure(open(cif_rnapolis.name, "r"))
    structure2d = annotator.extract_secondary_structure(structure3d)
    mapping = tertiary.Mapping2D3D(
        structure3d, structure2d.basePairs, structure2d.stackings, False
    )
    dbns = mapping._Mapping2D3D__generate_dot_bracket_per_strand(mapping.bpseq)
    i = 0
    struct3d=dict() # chain,seq_id,pairing

    for i, nucleotide in enumerate([i.chain+'.'+str(i.number)+'.'+i.name for i in structure3d.residues if i.is_nucleotide]):
        new_sequence=[]
        if ''.join(nucleotide.split('.')[:2]) in in_contact:
            if nucleotide.split('.')[0] not in struct3d.keys():
                struct3d[nucleotide.split('.')[0]]=[]
            struct3d[nucleotide.split('.')[0]].append([nucleotide.split('.')[1],nucleotide.split('.')[2],mapping.bpseq.dot_bracket.structure[i]])
        
    print(mapping.dot_bracket)
    print(struct3d)


>strand_T
AUCC
..((
>strand_P
GG
))
{'T': [['2', 'U', '.'], ['3', 'C', '('], ['4', 'C', '(']], 'P': [['1', 'G', ')'], ['2', 'G', ')']]}


In [71]:
print([i.chain+str(i.number) for i in structure3d.residues if i.is_nucleotide])
print(structure2d.singleStrands)
print(mapping.strands_sequences)
print(mapping)
print(mapping.bpseq.dot_bracket.structure)
structure3d.residues[4].is_nuleotide

['T1', 'T2', 'T3', 'T4', 'P1', 'P2']
[SingleStrand(strand=Strand(first=1, last=3, sequence='AUC', structure='..('), is5p=True, is3p=False)]
[('T', 'AUCC'), ('P', 'GG')]
Mapping2D3D(structure3d=Structure3D(residues=[T.A1, T.U2, T.C3, T.C4, T.HOH101, T.HOH102, T.HOH103, T.HOH104, T.HOH105, T.HOH106, T.HOH107, T.HOH108, T.HOH109, T.HOH110, P.G1, P.G2, P.HOH101, P.HOH102, P.HOH103, P.HOH104, P.HOH105], residue_map={ResidueLabel(chain='T', number=1, name='A'): T.A1, ResidueLabel(chain='T', number=2, name='U'): T.U2, ResidueLabel(chain='T', number=3, name='C'): T.C3, ResidueLabel(chain='T', number=4, name='C'): T.C4, ResidueLabel(chain='T', number=101, name='HOH'): T.HOH101, ResidueLabel(chain='T', number=102, name='HOH'): T.HOH102, ResidueLabel(chain='T', number=103, name='HOH'): T.HOH103, ResidueLabel(chain='T', number=104, name='HOH'): T.HOH104, ResidueLabel(chain='T', number=105, name='HOH'): T.HOH105, ResidueLabel(chain='T', number=106, name='HOH'): T.HOH106, ResidueLabel(chain='T', num

AttributeError: 'Residue3D' object has no attribute 'is_nuleotide'